# Simple Application with a Scikit-learn ML Model

One of the simplest yet typical problem that Machine Learning can help with in business is churn prediction. In this tutorial, we'll go through simple steps of solving this problem. We'll start with a problem definition, we'll do data exploratory analysis, prototype a model, and in the end, building an application that uses that model against the live data to help the sales department to plan their activities based on the predictions.

### Disclaimers

Note, in this tutorial there a lot of things, that in a real case are quite complicated, are simplified for the education purposes. Here's what is important:

1. In real life, data often resides in multiple data sources (including databases and datalakes) and needs additional cleanups and processing (often done using ETL solutions). In this tutorial, we'll use an already prepared dataset that emulates to a certain extent the data that is close to a real scenario.
2. When it comes to using ML model in a real situation, one of the most critical facts in the end is the acccuracy, reliability, and explainability of the model. In this tutorial, those questions will be only touched upon and will require from you additional dedicated work.
3. The target audience of this tutorials includes beginner data scientists that are only starting their data science careers interested in not just building an ML model but also putting it into production to drive day-to-day business decisions.

In this tutorial, we'll use `pandas` to work with data, `scikit-learn` to transform and train the model, and `dstack` to deploy the model and build the business application.


### Problem definition



### Exploring data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://www.dropbox.com/s/cat8vm6lchlu5tp/data.csv?dl=1", index_col=0)
df

,Company,Country,Region,Sector,Manager,y2015,y2016,y2017,y2018,y2019,RenewalMonth,RenewalDate,Churn
0,ICBC,China,Asia & Pacific,Financials,Edgar Livingston,0,0,0,1651,1423,4,2020-04-07,0.0
1,China Construction Bank,China,Asia & Pacific,Financials,Shawn Kirk,0,1774,1609,1624,1876,1,2020-01-21,0.0
2,Berkshire Hathaway,United States,North America,Financials,Alonzo Pierce,0,3286,3405,4082,4281,5,2020-05-01,0.0
3,JPMorgan Chase,United States,North America,Financials,Abram Gallagher,0,0,1671,1535,1806,8,2020-08-12,0.0
4,Wells Fargo,United States,North America,Financials,Eugene Graves,0,1362,1209,1203,1017,1,2020-01-13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,BEKB-BCBE,Switzerland,Europe,Financials,Darrell Frazier,8,7,8,7,6,2,2020-02-15,1.0
1996,Fastighets Balder,Sweden,Europe,Materials,Esther Knapp,0,0,0,8,8,1,2020-01-24,0.0
1997,Akamai Technologies,United States,North America,Information Technology,Eugene Graves,0,0,41,44,38,4,2020-04-25,0.0
1998,Oita Bank,Japan,Asia & Pacific,Financials,Frankie Valencia,10,11,12,10,8,7,2020-07-28,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       2000 non-null   object 
 1   Country       2000 non-null   object 
 2   Region        2000 non-null   object 
 3   Sector        1803 non-null   object 
 4   Manager       2000 non-null   object 
 5   y2015         2000 non-null   int64  
 6   y2016         2000 non-null   int64  
 7   y2017         2000 non-null   int64  
 8   y2018         2000 non-null   int64  
 9   y2019         2000 non-null   int64  
 10  RenewalMonth  2000 non-null   int64  
 11  RenewalDate   2000 non-null   object 
 12  Churn         1487 non-null   float64
dtypes: float64(1), int64(6), object(6)
memory usage: 218.8+ KB


In [4]:
df.describe()

,y2015,y2016,y2017,y2018,y2019,RenewalMonth,Churn
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,1487.000000
mean,103.134000,165.492500,206.979500,228.077000,264.29550,6.568000,0.178884
std,288.383992,367.342091,406.285524,432.801414,489.25395,3.446493,0.383384
min,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
25%,0.000000,0.000000,13.000000,31.000000,55.00000,4.000000,0.000000
50%,0.000000,45.000000,88.000000,104.000000,131.00000,7.000000,0.000000
75%,87.000000,165.000000,213.250000,230.000000,261.00000,10.000000,0.000000
max,3485.000000,4827.000000,5682.000000,6641.000000,9041.00000,12.000000,1.000000


In [5]:
train_df_1 = df[df["Churn"].notnull()]

In [6]:
train_df_1.groupby(["Country"])["Churn"].mean().sort_values(ascending=False)

Country
Czech Republic          1.000000
Peru                    1.000000
Nigeria                 0.500000
Mexico                  0.444444
Colombia                0.400000
Philippines             0.400000
Ireland                 0.357143
Qatar                   0.333333
Indonesia               0.333333
Bermuda                 0.333333
Portugal                0.333333
Brazil                  0.312500
Thailand                0.307692
Norway                  0.285714
Finland                 0.285714
Hong Kong               0.270833
China                   0.266667
Chile                   0.250000
Greece                  0.250000
South Korea             0.232558
Taiwan                  0.225806
Malaysia                0.222222
United States           0.194064
France                  0.186047
Singapore               0.181818
Germany                 0.177778
Italy                   0.166667
United Kingdom          0.161290
Australia               0.147059
Russia                  0.142857
Ca

In [7]:
train_df_1.groupby(["Country", "Sector"]).agg({'Company': 'size', 'Churn': 'mean'}).sort_values(ascending=False,
                                                                                                by='Churn')

Company  Churn
Country     Sector                                    
Canada      Information Technology            1    1.0
Greece      Utilities                         1    1.0
India       Industrials                       1    1.0
South Korea Utilities                         1    1.0
Sweden      Information Technology            1    1.0
...                                         ...    ...
Israel      Health Care                       1    0.0
            Information Technology            1    0.0
Italy       Financials                       10    0.0
            Telecommunication Services        1    0.0
Vietnam     Financials                        2    0.0

[264 rows x 2 columns]

### Feature engineering

In [8]:
def n_years(row):
    l = [row["y2019"], row["y2018"], row["y2017"], row["y2016"], row["y2015"]]
    return len([x for x in l if x != 0])


train_df_2 = train_df_1.copy()
train_df_2["Years"] = train_df_2.apply(n_years, axis=1)
train_df_2

,Company,Country,Region,Sector,Manager,y2015,y2016,y2017,y2018,y2019,RenewalMonth,RenewalDate,Churn,Years
0,ICBC,China,Asia & Pacific,Financials,Edgar Livingston,0,0,0,1651,1423,4,2020-04-07,0.0,2
1,China Construction Bank,China,Asia & Pacific,Financials,Shawn Kirk,0,1774,1609,1624,1876,1,2020-01-21,0.0,4
2,Berkshire Hathaway,United States,North America,Financials,Alonzo Pierce,0,3286,3405,4082,4281,5,2020-05-01,0.0,4
3,JPMorgan Chase,United States,North America,Financials,Abram Gallagher,0,0,1671,1535,1806,8,2020-08-12,0.0,3
4,Wells Fargo,United States,North America,Financials,Eugene Graves,0,1362,1209,1203,1017,1,2020-01-13,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,Aurubis,Germany,Europe,Materials,Nell Gibbs,112,104,104,109,127,8,2020-08-24,0.0,5
1995,BEKB-BCBE,Switzerland,Europe,Financials,Darrell Frazier,8,7,8,7,6,2,2020-02-15,1.0,5
1996,Fastighets Balder,Sweden,Europe,Materials,Esther Knapp,0,0,0,8,8,1,2020-01-24,0.0,2
1997,Akamai Technologies,United States,North America,Information Technology,Eugene Graves,0,0,41,44,38,4,2020-04-25,0.0,3


In [9]:
train_df_2.groupby(["Years"])["Churn"].mean().sort_values(ascending=False)

Years
1    0.411765
2    0.296875
3    0.213675
4    0.133739
5    0.084459
Name: Churn, dtype: float64

In [10]:
train_df_2.groupby(["Years", "Sector"]).agg({'Company': 'size', 'Churn': 'mean'}).sort_values(ascending=False,
                                                                                              by='Churn')

Company     Churn
Years Sector                                       
1     Energy                            7  0.857143
      Information Technology           18  0.666667
      Consumer Discretionary           20  0.600000
2     Telecommunication Services        5  0.600000
1     Telecommunication Services        7  0.571429
      Industrials                      18  0.555556
      Utilities                        12  0.500000
2     Consumer Discretionary           19  0.473684
      Energy                           10  0.400000
1     Consumer Staples                 10  0.400000
2     Industrials                      15  0.400000
3     Industrials                      30  0.366667
2     Consumer Staples                  9  0.333333
      Health Care                       6  0.333333
1     Materials                        15  0.333333
3     Information Technology           21  0.333333
4     Telecommunication Services       10  0.300000
2     Materials                        14  0.285714
1     Health Care                      14  0.285714
3     Utilities                        14  0.285714
4     Industrials                      36  0.277778
5     Information Technology           26  0.269231
3     Materials                        24  0.250000
4     Information Technology           20  0.250000
3     Consumer Discretionary           31  0.225806
      Consumer Staples                  5  0.200000
      Telecommunication Services        6  0.166667
1     Financials                       67  0.164179
5     Materials                        45  0.133333
4     Consumer Discretionary           31  0.129032
5     Energy                           32  0.125000
4     Utilities                        16  0.125000
3     Energy                           17  0.117647
5     Consumer Discretionary           82  0.109756
      Consumer Staples                 37  0.108108
3     Financials                       57  0.105263
2     Financials                       30  0.100000
4     Financials                      116  0.094828
      Consumer Staples                 13  0.076923
      Health Care                      13  0.076923
5     Telecommunication Services       14  0.071429
      Industrials                      63  0.063492
      Utilities                        24  0.041667
4     Materials                        26  0.038462
5     Financials                      168  0.035714
      Health Care                      31  0.032258
4     Energy                           14  0.000000
3     Health Care                       9  0.000000
2     Utilities                         7  0.000000
      Information Technology            6  0.000000

In [11]:
train_df_2 = train_df_1.copy()
train_df_2 = train_df_2.drop(["Company", "Region", "Manager", "RenewalMonth", "RenewalDate"], axis=1)
train_df_2

,Country,Sector,y2015,y2016,y2017,y2018,y2019,Churn
0,China,Financials,0,0,0,1651,1423,0.0
1,China,Financials,0,1774,1609,1624,1876,0.0
2,United States,Financials,0,3286,3405,4082,4281,0.0
3,United States,Financials,0,0,1671,1535,1806,0.0
4,United States,Financials,0,1362,1209,1203,1017,0.0
...,...,...,...,...,...,...,...,...
1994,Germany,Materials,112,104,104,109,127,0.0
1995,Switzerland,Financials,8,7,8,7,6,1.0
1996,Sweden,Materials,0,0,0,8,8,0.0
1997,United States,Information Technology,0,0,41,44,38,0.0


### Data normalization

In [170]:
train_df = train_df_2.copy()
for col in ["y2015", "y2016", "y2017", "y2018", "y2019"]:
    train_df[col] = train_df[col] / train_df[col].max()

for c in train_df_2["Country"].unique():
    train_df[c] = train_df["Country"].apply(lambda x: 1 if x == c else 0)

for s in train_df_2["Sector"].unique():
    if s:
        train_df[s] = train_df["Sector"].apply(lambda x: 1 if x == s else 0)

train_df = train_df.drop(["Country", "Sector"], axis=1)

train_df

,y2015,y2016,y2017,y2018,y2019,Churn,China,United States,South Korea,Germany,...,Information Technology,Telecommunication Services,Energy,Industrials,Consumer Discretionary,Health Care,Consumer Staples,Materials,Utilities,NaN
0,0.000000,0.000000,0.000000,0.248607,0.157394,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.367516,0.283175,0.244541,0.207499,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.680754,0.599261,0.614666,0.473510,0.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.000000,0.294087,0.231140,0.199757,0.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,0.282163,0.212777,0.181147,0.112488,0.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,0.032138,0.021545,0.018303,0.016413,0.014047,0.0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1995,0.002296,0.001450,0.001408,0.001054,0.000664,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,0.000000,0.000000,0.000000,0.001205,0.000885,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1997,0.000000,0.000000,0.007216,0.006626,0.004203,0.0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


### Training model

In [198]:
from sklearn.model_selection import train_test_split

In [199]:
X = train_df.drop(["Churn"], axis=1)
y = train_df["Churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040 entries, 1759 to 841
Data columns (total 74 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   y2015                       1040 non-null   float64
 1   y2016                       1040 non-null   float64
 2   y2017                       1040 non-null   float64
 3   y2018                       1040 non-null   float64
 4   y2019                       1040 non-null   float64
 5   China                       1040 non-null   int64  
 6   United States               1040 non-null   int64  
 7   South Korea                 1040 non-null   int64  
 8   Germany                     1040 non-null   int64  
 9   Hong Kong                   1040 non-null   int64  
 10  France                      1040 non-null   int64  
 11  Japan                       1040 non-null   int64  
 12  Spain                       1040 non-null   int64  
 13  Switzerland                 104

In [200]:
from sklearn.linear_model import LogisticRegression

In [202]:
model = LogisticRegression()
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(accuracy * 100, '%')
# summarize feature importance
for i, v in enumerate(model.coef_[0]):
    if v > 0.5:
        print('Feature: %0s, Score: %.5f' % (X_train.columns[i], v))

81.20805369127517 %
Feature: y2019, Score: 0.89807
Feature: Brazil, Score: 0.54323
Feature: Ireland, Score: 0.91486
Feature: Thailand, Score: 0.54994
Feature: Mexico, Score: 0.50326
Feature: Finland, Score: 0.59626
Feature: Colombia, Score: 0.81784
Feature: Peru, Score: 0.80034
Feature: Qatar, Score: 0.77438
Feature: Czech Republic, Score: 0.70177
Feature: Portugal, Score: 0.79294
Feature: Information Technology, Score: 0.58487
Feature: Industrials, Score: 0.55374


### Making the model re-usable

In [203]:
live_df = df[df["Churn"].isnull()]
live_df = live_df.drop(["Churn"], axis=1)
live_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 7 to 1999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company       513 non-null    object
 1   Country       513 non-null    object
 2   Region        513 non-null    object
 3   Sector        463 non-null    object
 4   Manager       513 non-null    object
 5   y2015         513 non-null    int64 
 6   y2016         513 non-null    int64 
 7   y2017         513 non-null    int64 
 8   y2018         513 non-null    int64 
 9   y2019         513 non-null    int64 
 10  RenewalMonth  513 non-null    int64 
 11  RenewalDate   513 non-null    object
dtypes: int64(6), object(6)
memory usage: 52.1+ KB


In [204]:
from sklearn.base import BaseEstimator, TransformerMixin

In [205]:
class PrepareData(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def transform(self, X, **transform_params):
        X_prepared = X.copy()

        def n_years(row):
            l = [row["y2019"], row["y2018"], row["y2017"], row["y2016"], row["y2015"]]
            return len([x for x in l if x != 0])

        X_prepared["Years"] = X_prepared.apply(n_years, axis=1)

        X_prepared = X_prepared.drop(["Company", "Region", "Manager", "RenewalMonth", "RenewalDate"], axis=1)

        for col in ["y2015", "y2016", "y2017", "y2018", "y2019"]:
            X_prepared[col] = X_prepared[col] / X_prepared[col].max()

        for c in X["Country"].unique():
            X_prepared[c] = X_prepared["Country"].apply(lambda x: 1 if x == c else 0)

        for s in X["Sector"].unique():
            if s:
                X_prepared[s] = X_prepared["Sector"].apply(lambda x: 1 if x == s else 0)

        X_prepared = X_prepared.drop(["Country", "Sector"], axis=1)
        return X_prepared

    def fit(self, X, y=None, **fit_params):
        return self

In [206]:
prepared_df = PrepareData().transform(live_df)
prepared_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 7 to 1999
Data columns (total 70 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   y2015                       513 non-null    float64
 1   y2016                       513 non-null    float64
 2   y2017                       513 non-null    float64
 3   y2018                       513 non-null    float64
 4   y2019                       513 non-null    float64
 5   Years                       513 non-null    int64  
 6   China                       513 non-null    int64  
 7   Japan                       513 non-null    int64  
 8   Netherlands                 513 non-null    int64  
 9   France                      513 non-null    int64  
 10  United States               513 non-null    int64  
 11  Canada                      513 non-null    int64  
 12  India                       513 non-null    int64  
 13  Switzerland                 513 no

In [185]:
# predicted_churn = model.predict(prepared_df)  # ValueError: X has 70 features per sample; expecting 74

In [207]:
class ReindexColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def transform(self, X, **transform_params):
        return X.reindex(columns=self.columns, fill_value=0)

    def fit(self, X, y=None, **fit_params):
        return self

In [208]:
reindexed_df = ReindexColumns(X.columns).transform(prepared_df)
reindexed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 7 to 1999
Data columns (total 74 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   y2015                       513 non-null    float64
 1   y2016                       513 non-null    float64
 2   y2017                       513 non-null    float64
 3   y2018                       513 non-null    float64
 4   y2019                       513 non-null    float64
 5   China                       513 non-null    int64  
 6   United States               513 non-null    int64  
 7   South Korea                 513 non-null    int64  
 8   Germany                     513 non-null    int64  
 9   Hong Kong                   513 non-null    int64  
 10  France                      513 non-null    int64  
 11  Japan                       513 non-null    int64  
 12  Spain                       513 non-null    int64  
 13  Switzerland                 513 no

In [209]:
from scipy.stats import describe

In [210]:
predicted_churn = model.predict(reindexed_df)
describe(predicted_churn)

DescribeResult(nobs=513, minmax=(0.0, 1.0), mean=0.001949317738791423, variance=0.0019493177387914218, skewness=22.583222824145377, kurtosis=508.0019531250005)

In [211]:
from sklearn.pipeline import Pipeline

In [212]:
pipeline = Pipeline([
    ('prepare', PrepareData()),
    ('reindex', ReindexColumns(X.columns)),
    ('regression', LogisticRegression())
])
X_1 = df[df["Churn"].notnull()]
y_1 = X_1["Churn"]
X_1 = X_1.drop(["Churn"], axis=1)
pipeline.fit(X_1, y_1)
predicted_churn = pipeline.predict(live_df)
describe(predicted_churn)

DescribeResult(nobs=513, minmax=(0.0, 1.0), mean=0.003898635477582846, variance=0.0038910209551656924, skewness=15.921806238436796, kurtosis=251.5039138943249)

### Intro to dstack


### Deploying the model to dstack

### Building the application